In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import scipy as scipy
from scipy import special, optimize
import seaborn as sns
%matplotlib inline

resp = pd.read_csv('../project/data/ehresp_2014/ehresp_2014.csv')
resp.head()


,tucaseid,tulineno,eeincome1,erbmi,erhhch,erincome,erspemch,ertpreat,ertseat,ethgt,...,eumeat,eumilk,euprpmel,eusoda,eustores,eustreason,eutherm,euwgt,euwic,exincome1
0,20140101140007,1,-2,33.200001,1,-1,-1,30,2,0,...,1,2,1,-1,2,1,2,170,1,2
1,20140101140011,1,1,22.700001,3,1,-1,45,14,0,...,1,2,1,-1,1,2,2,128,2,0
2,20140101140028,1,2,49.400002,3,5,-1,60,0,0,...,-1,-1,2,2,-1,-1,-1,270,2,12
3,20140101140063,1,-2,-1.000000,3,-1,-1,0,0,0,...,2,2,1,1,2,6,-1,-2,2,2
4,20140101140168,1,2,31.000000,3,5,-1,65,0,0,...,1,2,1,2,1,1,2,210,1,0


In [2]:
print resp.columns
# print resp.describe()

Index([u'tucaseid', u'tulineno', u'eeincome1', u'erbmi', u'erhhch',
       u'erincome', u'erspemch', u'ertpreat', u'ertseat', u'ethgt', u'etwgt',
       u'eudietsoda', u'eudrink', u'eueat', u'euexercise', u'euexfreq',
       u'eufastfd', u'eufastfdfrq', u'euffyday', u'eufdsit', u'eufinlwgt',
       u'eusnap', u'eugenhth', u'eugroshp', u'euhgt', u'euinclvl',
       u'euincome2', u'eumeat', u'eumilk', u'euprpmel', u'eusoda', u'eustores',
       u'eustreason', u'eutherm', u'euwgt', u'euwic', u'exincome1'],
      dtype='object')


In [3]:
resp.rename(columns={'erbmi':'bmi', 
                     'ertpreat':'primary_eating', 
                     'ertseat':'secondary_eating', 
                     'eudietsoda':'drink_dietsoda', 
                     'eudrink':'drink', 
                     'eueat':'eat', 
                     'euexercise':'exercise', 
                     'euexfreq':'exercise_freq',
                     'eufastfd':'fast_food',
                     'eufastfdfrq':'fast_food_freq',
                     'eufdsit':'food_amount',
                     'euinclvl':'income_14_15',
                     'euincome2':'income_130',
                     'eusoda':'drink_soda',
                     'eustores':'stores'
                     },
            inplace=True)

                     

In [4]:
resp.drop(['eeincome1', 
           'erhhch', 
           'erincome', 
           'erspemch', 
           'ethgt', 
           'etwgt', 
           'euffyday', 
           'eufinlwgt', 
           'eusnap', 
           'eugenhth',
           'eugroshp',
           'euhgt',
           'eumeat',
           'eumilk',
           'euprpmel',
           'eustreason',
           'eutherm',
           'euwic'
          ], axis=1, inplace=True)


In [5]:
resp.head()

,tucaseid,tulineno,bmi,primary_eating,secondary_eating,drink_dietsoda,drink,eat,exercise,exercise_freq,fast_food,fast_food_freq,food_amount,income_14_15,income_130,drink_soda,stores,euwgt,exincome1
0,20140101140007,1,33.200001,30,2,-1,2,1,2,-1,2,-1,1,5,-2,-1,2,170,2
1,20140101140011,1,22.700001,45,14,-1,2,1,2,-1,1,1,1,5,-1,-1,1,128,0
2,20140101140028,1,49.400002,60,0,-1,1,2,2,-1,2,-1,1,5,2,2,-1,270,12
3,20140101140063,1,-1.000000,0,0,2,1,2,2,-1,2,-1,1,5,-2,1,2,-2,2
4,20140101140168,1,31.000000,65,0,-1,1,2,1,5,2,-1,1,5,2,2,1,210,0


In [7]:
resp.income_14_15.value_counts()

5    9232
6    1980
Name: income_14_15, dtype: int64

In [8]:
resp_df = resp.drop(['tucaseid', 'tulineno'], axis=1)

In [9]:
resp_df.head()

,bmi,primary_eating,secondary_eating,drink_dietsoda,drink,eat,exercise,exercise_freq,fast_food,fast_food_freq,food_amount,income_14_15,income_130,drink_soda,stores,euwgt,exincome1
0,33.200001,30,2,-1,2,1,2,-1,2,-1,1,5,-2,-1,2,170,2
1,22.700001,45,14,-1,2,1,2,-1,1,1,1,5,-1,-1,1,128,0
2,49.400002,60,0,-1,1,2,2,-1,2,-1,1,5,2,2,-1,270,12
3,-1.000000,0,0,2,1,2,2,-1,2,-1,1,5,-2,1,2,-2,2
4,31.000000,65,0,-1,1,2,1,5,2,-1,1,5,2,2,1,210,0


In [10]:
resp_df_14 = resp_df[resp_df['income_14_15'] == 5]
resp_df_15 = resp_df[resp_df['income_14_15'] == 6]


In [11]:
resp_df_14.head()

,bmi,primary_eating,secondary_eating,drink_dietsoda,drink,eat,exercise,exercise_freq,fast_food,fast_food_freq,food_amount,income_14_15,income_130,drink_soda,stores,euwgt,exincome1
0,33.200001,30,2,-1,2,1,2,-1,2,-1,1,5,-2,-1,2,170,2
1,22.700001,45,14,-1,2,1,2,-1,1,1,1,5,-1,-1,1,128,0
2,49.400002,60,0,-1,1,2,2,-1,2,-1,1,5,2,2,-1,270,12
3,-1.000000,0,0,2,1,2,2,-1,2,-1,1,5,-2,1,2,-2,2
4,31.000000,65,0,-1,1,2,1,5,2,-1,1,5,2,2,1,210,0


In [12]:
resp_df_14.rename(columns={'income_14_15':'income_14',}, inplace=True)
resp_df_15.rename(columns={'income_14_15':'income_15',}, inplace=True)

/Users/FrancescaSchott/anaconda/envs/testenv/lib/python2.7/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [13]:
resp_df_15.head()

,bmi,primary_eating,secondary_eating,drink_dietsoda,drink,eat,exercise,exercise_freq,fast_food,fast_food_freq,food_amount,income_15,income_130,drink_soda,stores,euwgt,exincome1
8503,-1.000000,0,20,2,1,1,1,5,1,12,1,6,-2,1,-1,-2,75
8566,22.100000,50,5,1,1,1,1,1,1,4,1,6,-1,1,1,150,0
8569,20.200001,0,20,-1,1,1,2,-1,1,4,1,6,-1,2,-1,129,0
8577,33.000000,10,0,-1,1,2,1,2,2,-1,1,6,-1,2,-1,230,0
8593,26.600000,10,0,2,1,2,2,-1,1,2,2,6,2,1,2,160,0


In [14]:
resp_df.income_130.value_counts()

-1    6818
 2    2038
 1    1116
-2     599
 3     359
-3     282
Name: income_130, dtype: int64